In [3]:
from datasets import load_dataset
import numpy as np
import datasets
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
# Download and load the Cohere dataset form Hugging face
# cohere_dataset = datasets.load_dataset("Cohere/wikipedia-22-12", 'en')

# as The dataset is very big. So I have downloaded a protion of it to work on it.

cohere_dataset = load_dataset ('json', data_files = r"C:\Users\Z004RJZU\Documents\LLM_Practice\LLM\Datasets\Cohere\000.jsonl", split='train')

In [3]:
cohere_dataset

Dataset({
    features: ['wiki_id', 'url', 'views', 'langs', 'title', 'text', 'paragraph_id', 'id'],
    num_rows: 100000
})

In [4]:
cohere_dataset[:3]

{'wiki_id': [69407798, 3524766, 3524766],
 'url': ['https://en.wikipedia.org/wiki?curid=69407798',
  'https://en.wikipedia.org/wiki?curid=3524766',
  'https://en.wikipedia.org/wiki?curid=3524766'],
 'views': [5674.4492597435465, 5409.5609619796405, 5409.5609619796405],
 'langs': [38, 184, 184],
 'title': ['Deaths in 2022', 'YouTube', 'YouTube'],
 'text': ['The following notable deaths occurred in 2022. Names are reported under the date of death, in alphabetical order. A typical entry reports information in the following sequence:',
  'YouTube is a global online video sharing and social media platform headquartered in San Bruno, California. It was launched on February 14, 2005, by Steve Chen, Chad Hurley, and Jawed Karim. It is owned by Google, and is the second most visited website, after Google Search. YouTube has more than 2.5 billion monthly users who collectively watch more than one billion hours of videos each day. , videos were being uploaded at a rate of more than 500 hours of c

In [5]:
# Removing unnecessory columns from dataset
cohere_dataset = cohere_dataset.select_columns ([ 'url',  'title', 'text' ])
cohere_dataset

Dataset({
    features: ['url', 'title', 'text'],
    num_rows: 100000
})

In [6]:
cohere_dataset_text_len = cohere_dataset.map(lambda x : {'text_len' : len(x['text']) })
cohere_dataset_text_len

Dataset({
    features: ['url', 'title', 'text', 'text_len'],
    num_rows: 100000
})

In [7]:
cohere_dataset_text_len[0]

{'url': 'https://en.wikipedia.org/wiki?curid=69407798',
 'title': 'Deaths in 2022',
 'text': 'The following notable deaths occurred in 2022. Names are reported under the date of death, in alphabetical order. A typical entry reports information in the following sequence:',
 'text_len': 176}

In [8]:
cmb_dataset = cohere_dataset_text_len. map (lambda x : {'text_comp' :  x[ 'title' ] + " \n " + x [ 'text' ] + " \n " + x [ 'url' ] }, ).select_columns('text_comp')
cmb_dataset

Dataset({
    features: ['text_comp'],
    num_rows: 100000
})

In [9]:
cmb_dataset[0]

{'text_comp': 'Deaths in 2022 \n The following notable deaths occurred in 2022. Names are reported under the date of death, in alphabetical order. A typical entry reports information in the following sequence: \n https://en.wikipedia.org/wiki?curid=69407798'}

In [10]:
True in (np.array( cohere_dataset_text_len['text_len']) < 25)

False

As not text is small then 25 words so I dont need to remove small paragraphs from dataset 

### Tockenizing the dataset

In [4]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained (model_ckpt)

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [58]:
from transformers import DataCollatorWithPadding 
datacollector =  DataCollatorWithPadding (model_ckpt, padding=True, return_tensors='pt')

In [7]:
def get_embedding(text):
    encoded_text = tokenizer ( text=text, truncation=True,padding=True, return_tensors='pt' ).to(device)
    model.to (device)
    encodeed_last_out_state = model(** encoded_text).last_hidden_state[:,0]
    return encodeed_last_out_state



In [109]:
data = get_embedding ( cohere_dataset_text_len['text'][0] )
data

tensor([[ 4.9356e-01,  3.2731e-01, -2.3691e-01,  3.4945e-01, -1.0965e-01,
         -2.7030e-01,  3.6670e-02,  4.9790e-02, -2.8614e-01,  2.8275e-01,
          2.5917e-01,  2.7235e-01,  1.5068e-01, -1.3791e-01, -1.4966e-02,
         -4.1428e-01,  3.3530e-01, -1.3885e-01, -2.5672e-01,  2.2994e-01,
         -2.8966e-01,  4.6551e-01, -3.0376e-01, -1.1332e-01, -9.9217e-02,
          2.6644e-01, -3.4776e-02,  1.5970e-01,  1.4803e-01, -3.3811e-01,
         -2.6734e-01, -1.5671e-01,  1.9524e-04, -1.8536e-01, -1.0521e-04,
         -3.2258e-01,  3.0876e-01, -8.5313e-02, -3.4510e-01,  9.0497e-02,
         -3.4652e-01, -2.5919e-01, -1.5309e-01, -2.6753e-01,  4.1939e-03,
         -4.6443e-01, -1.1720e-01, -1.0840e-02,  9.7699e-02,  5.7875e-02,
          2.8734e-01, -2.1094e-01,  4.7801e-01, -3.7467e-01,  1.9660e-01,
          2.3358e-02, -2.2511e-01, -1.6760e-01,  1.5551e-01, -2.6801e-01,
         -4.3818e-02,  2.8158e-01,  1.9379e-02, -1.3384e-01,  3.2284e-01,
         -1.8912e-03, -2.0879e-03,  2.

This below work is done to check what kind of functionality does a fucntion provides

In [85]:
text = cohere_dataset_text_len['text'][0]
encoded_text_0 = tokenizer ( text=text, truncation=True,padding=True, return_tensors='pt' ).to(device=device)
encoded_text_0

{'input_ids': tensor([[    0,  2000,  2210,  3866,  6681,  4162,  2003, 16802,  2479,  1016,
          3419,  2028,  2992,  2108,  2000,  3062,  2001,  2335,  1014,  2003,
         12444,  7480,  2348,  1016,  1041,  5175,  4447,  4315,  2596,  2003,
          2000,  2210,  5541,  1028,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [110]:
model.to (device)
temp = model(**encoded_text_0).last_hidden_state[:,0]
temp , temp.shape


(tensor([[ 4.9356e-01,  3.2731e-01, -2.3691e-01,  3.4945e-01, -1.0965e-01,
          -2.7030e-01,  3.6670e-02,  4.9790e-02, -2.8614e-01,  2.8275e-01,
           2.5917e-01,  2.7235e-01,  1.5068e-01, -1.3791e-01, -1.4966e-02,
          -4.1428e-01,  3.3530e-01, -1.3885e-01, -2.5672e-01,  2.2994e-01,
          -2.8966e-01,  4.6551e-01, -3.0376e-01, -1.1332e-01, -9.9217e-02,
           2.6644e-01, -3.4776e-02,  1.5970e-01,  1.4803e-01, -3.3811e-01,
          -2.6734e-01, -1.5671e-01,  1.9524e-04, -1.8536e-01, -1.0521e-04,
          -3.2258e-01,  3.0876e-01, -8.5313e-02, -3.4510e-01,  9.0497e-02,
          -3.4652e-01, -2.5919e-01, -1.5309e-01, -2.6753e-01,  4.1939e-03,
          -4.6443e-01, -1.1720e-01, -1.0840e-02,  9.7699e-02,  5.7875e-02,
           2.8734e-01, -2.1094e-01,  4.7801e-01, -3.7467e-01,  1.9660e-01,
           2.3358e-02, -2.2511e-01, -1.6760e-01,  1.5551e-01, -2.6801e-01,
          -4.3818e-02,  2.8158e-01,  1.9379e-02, -1.3384e-01,  3.2284e-01,
          -1.8912e-03, -2

In [17]:
embedded_dataset = cmb_dataset.map (lambda x: {'tenser': get_embedding(x['text_comp']).detach().cpu().numpy()[0] } )
embedded_dataset

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [24]:
text[0]

In [25]:
embedded_dataset.save_to_disk("Cohere_tokenized")

In [2]:
import datasets

cohere_dataset = datasets.load_from_disk("Cohere_tokenized")
cohere_dataset[0]

{'text_comp': 'Deaths in 2022 \n The following notable deaths occurred in 2022. Names are reported under the date of death, in alphabetical order. A typical entry reports information in the following sequence: \n https://en.wikipedia.org/wiki?curid=69407798',
 'tenser': [0.4517451226711273,
  0.3878551423549652,
  -0.24963058531284332,
  0.21310476958751678,
  -0.009713918901979923,
  -0.3349539339542389,
  -0.08604103326797485,
  0.2057056576013565,
  -0.4503140151500702,
  0.3566235303878784,
  0.246233731508255,
  0.3311823308467865,
  0.12147241830825806,
  -0.0007445981609635055,
  0.15428389608860016,
  -0.3574395477771759,
  0.5098404288291931,
  -0.3457687497138977,
  -0.18911772966384888,
  0.18914729356765747,
  -0.44129085540771484,
  0.5595310926437378,
  -0.2678220868110657,
  -0.1702195256948471,
  -0.05536064878106117,
  0.2569497227668762,
  -0.10870569944381714,
  0.01616617850959301,
  0.12205243110656738,
  -0.3870489001274109,
  0.042172547429800034,
  -0.2708313763

In [13]:
cohere_dataset.add_faiss_index(column='tenser')

  0%|          | 0/100 [00:00<?, ?it/s]

Dataset({
    features: ['text_comp', 'tenser'],
    num_rows: 100000
})

In [8]:
question = "what is youtube?"

In [12]:
q_tok = get_embedding([question]).detach().cpu().numpy()
q_tok.shape

(1, 768)

In [14]:
score, samples = cohere_dataset.get_nearest_examples('tenser', q_tok,k=5)

In [18]:
score, samples

(array([19.084007, 20.072508, 21.04601 , 21.06257 , 21.555721],
       dtype=float32),
 {'text_comp': ['YouTube \n YouTube Movies is a service by YouTube that shows movies via its website. Around 100–500 movies are free to view, with ads. Some new movies get added and some get removed, unannounced at a new month. \n https://en.wikipedia.org/wiki?curid=3524766',
   'YouTube Music \n The availability of music includes many of the releases from mainstream artists and extends to any video categorized as music on the YouTube service. \n https://en.wikipedia.org/wiki?curid=52135003',
   'YouTube \n Later the same year, "YouTube Feather" was introduced as a lightweight alternative website for countries with limited internet speeds. \n https://en.wikipedia.org/wiki?curid=3524766',
   'YouTube \n YouTube is a global online video sharing and social media platform headquartered in San Bruno, California. It was launched on February 14, 2005, by Steve Chen, Chad Hurley, and Jawed Karim. It is owned

In [20]:
print ("Results against your serach are \n" + samples["text_comp"][-1] + "\n\n" + samples["text_comp"][-2])

Results against your serach are 
YouTube Music 
 YouTube Music is a music streaming service developed by YouTube, a subsidiary of Google. It provides a tailored interface for the service, oriented towards music streaming, allowing users to browse through songs and music videos on YouTube based on genres, playlists, and recommendations. 
 https://en.wikipedia.org/wiki?curid=52135003

YouTube 
 YouTube is a global online video sharing and social media platform headquartered in San Bruno, California. It was launched on February 14, 2005, by Steve Chen, Chad Hurley, and Jawed Karim. It is owned by Google, and is the second most visited website, after Google Search. YouTube has more than 2.5 billion monthly users who collectively watch more than one billion hours of videos each day. , videos were being uploaded at a rate of more than 500 hours of content per minute. 
 https://en.wikipedia.org/wiki?curid=3524766


# Lets make a funciton that can Use Grdio to answer our questions.